In [1]:
cd /data/trungnv/Face_Recognition/

/data/trungnv/Face_Recognition


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2022-08-09 23:27:07.528948: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
from __future__ import print_function, division

import torch
import torch.onnx
import time
import onnx
import onnxruntime

import net
import numpy as np
import cv2
import tensorrt as trt

import pycuda.driver as cuda
import pycuda.autoinit

import tensorrt as trt


In [4]:
adaface_models = {
    'ir_101': "/data/trungnv/Face_Recognition/finetune/finetune_ir101_webface12m.ckpt",
}

architecture = 'ir_101'
assert architecture in adaface_models.keys()
model_pytorch = net.build_model(architecture)
statedict = torch.load(adaface_models[architecture])['state_dict']
model_statedict = {key[6:]: val for key, val in statedict.items() if key.startswith('model.')}
model_pytorch.load_state_dict(model_statedict)
model_pytorch.eval()
model_pytorch.cuda()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [5]:
# input
dummy_input=torch.randn(4, 3, 112, 112).cuda()

# export the model to ONNX
torch.onnx.export(model_pytorch, dummy_input, "finetune_ir101_webface12m_gpu.onnx", verbose=False)

# export model ONNX gpu to TensorRT gpu
TRT_LOGGER =trt.Logger(trt.Logger.WARNING)
EXPLICIT_BATCH = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)

builder = trt.Builder(TRT_LOGGER)
config = builder.create_builder_config()
network = builder.create_network(EXPLICIT_BATCH)
parser = trt.OnnxParser(network, TRT_LOGGER)

with open('finetune_ir101_webface12m_gpu.onnx', 'rb') as model:
    if not parser.parse(model.read()):
        for error in range(parser.num_errors):
            print(parser.get_error(error)) 

engine = builder.build_engine(network, config)

engine_filename = "finetune_ir101_webface12m_gpu.trt"

if engine_filename:
    with open(engine_filename, 'wb') as f:
        f.write(engine.serialize())

[TensorRT] WARNING: onnx2trt_utils.cpp:220: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing the workspace size to 4194304 bytes to get better performance.
[TensorRT] WARNING: Try increasing 

In [6]:
# load model TensorRT gpu
from onnx_helper import ONNXClassifierWrapper
PRECISION = np.float32
model_tensorrt = ONNXClassifierWrapper("finetune_ir101_webface12m_gpu.trt", [1, 512], target_dtype = PRECISION)